In [0]:
import functools
import numpy as np
import tensorflow as tf

In [0]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [0]:
np.set_printoptions(precision=3, suppress=True)

In [0]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [0]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)


In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key, value.numpy()))

In [33]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'female']
age                 : [27. 17. 18. 20. 22.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [2 0 0 0 2]
fare                : [211.5     8.663   9.842   9.5    49.5  ]
class               : [b'First' b'Third' b'Third' b'Third' b'First']
deck                : [b'C' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [34]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [39. 36.  7. 31.  2.]
n_siblings_spouses  : [0 0 0 1 4]
class               : [b'Second' b'Third' b'Second' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'D' b'unknown']
alone               : [b'y' b'y' b'n' b'n' b'n']


In [36]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)
show_batch(temp_dataset)

age                 : [18.  2. 19. 47. 28.]
n_siblings_spouses  : [0. 1. 0. 0. 0.]
parch               : [2. 1. 0. 0. 0.]
fare                : [13.    26.     8.158 15.    79.2  ]


In [0]:
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()),axis=-1 ), label


In [43]:
packed_dataset = temp_dataset.map(pack)
for features, labels in packed_dataset.take(1):
  print("feature: ")
  print(features.numpy())
  print("label:", labels.numpy())

feature: 
[[ 18.      2.      2.    262.375]
 [ 27.      0.      0.      7.896]
 [ 18.      0.      1.     23.   ]
 [ 17.      0.      0.     12.   ]
 [ 24.      0.      0.      8.05 ]]
label: [1 0 1 1 0]


In [44]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'female' b'male']
age                 : [28.  2. 21. 26. 28.]
n_siblings_spouses  : [0 3 0 0 0]
parch               : [0 1 0 0 0]
fare                : [ 7.75  21.075  8.05  78.85   7.55 ]
class               : [b'Third' b'Third' b'Third' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [0]:
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
class PackNumericFeatures(object):
  def __init__(seft, names):
    seft.names = names
  
  def __call__(seft, features, labels):
    numeric_features = [features.pop(name) for name in seft.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features
    return features, labels

  

In [0]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']
packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES)
)
packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES)
)

In [49]:
show_batch(packed_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'female']
class               : [b'Second' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'y']
numeric             : [[27.     1.     0.    21.   ]
 [28.     0.     0.     8.05 ]
 [28.     0.     0.     7.896]
 [ 9.     1.     1.    15.9  ]
 [37.     0.     0.     9.587]]


In [0]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [52]:
import pandas as pd 
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std
  

In [56]:
normalizer = functools.partial(normalize_numeric_data, mean = MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn = normalizer, shape = [len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fc9af54ea60>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [57]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 6.   ,  0.   ,  1.   , 12.475],
       [40.   ,  1.   ,  0.   ,  9.475],
       [28.   ,  0.   ,  0.   ,  7.225],
       [35.   ,  0.   ,  0.   ,  8.05 ],
       [30.   ,  0.   ,  0.   ,  8.663]], dtype=float32)>

In [59]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-1.889, -0.474,  0.782, -0.401],
       [ 0.829,  0.395, -0.479, -0.456],
       [-0.13 , -0.474, -0.479, -0.497],
       [ 0.429, -0.474, -0.479, -0.482],
       [ 0.029, -0.474, -0.479, -0.471]], dtype=float32)

In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [62]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [68]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [70]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     1.
  0.     0.     0.     0.     0.     0.     0.     0.    -1.889 -0.474
  0.782 -0.401  1.     0.   ]


In [71]:
model = tf.keras.Sequential([
                             preprocessing_layer,
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(1),
])

model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = 'adam',
    metrics = ['accuracy']
)

ERROR! Session/line number was not unique in database. History logging moved to new session 65


In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [73]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 2ms/step - loss: 0.5076 - accuracy: 0.7321
Epoch 2/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4212 - accuracy: 0.8150
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4075 - accuracy: 0.8198
Epoch 4/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3907 - accuracy: 0.8373
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3850 - accuracy: 0.8278
Epoch 6/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3753 - accuracy: 0.8389
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3604 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3624 - accuracy: 0.8373
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3537 - accuracy: 0.8421
Epoch 10/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3477 - accuracy: 0.8533

In [77]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\nTest Loss {}, Test Accuracy {:5.2f}%'.format(test_loss, 100*test_accuracy))


53/53 [==============================] - 0s 1ms/step - loss: 0.4497 - accuracy: 0.8371

Test Loss 0.4496898651123047, Test Accuracy 83.71%


In [78]:
predictions = model.predict(test_data)
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2f}".format(prediction[0]),
        "| Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 0.79 | Actual outcome:  DIED
Predicted survival: 0.01 | Actual outcome:  DIED
Predicted survival: 0.00 | Actual outcome:  DIED
Predicted survival: 0.07 | Actual outcome:  DIED
Predicted survival: 0.79 | Actual outcome:  SURVIVED
